# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "human_nontata_promoters"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_nontata_promoters 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
2022-06-03 06:59:46.067131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-03 06:59:46.458639: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43066 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:a3:00.0, compute capability: 8.6


In [5]:
info(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lengths of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
NUM_CLASSES = len(CLASSES)

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


In [7]:
if NUM_CLASSES > 2:
    train_dset = train_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

## Text vectorization

In [8]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())
vectorize_layer.get_vocabulary()

['', '[UNK]', 'g', 'c', 't', 'a']

In [9]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [10]:
model = get_model(NUM_CLASSES, VOCAB_SIZE)

In [11]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


2022-06-03 06:59:59.748623: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204
2022-06-03 07:00:01.179586: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


424/424 [==============================] - 10s 16ms/step - loss: 0.4403 - binary_accuracy: 0.8035 - f1_score: 0.7488
Epoch 2/10
424/424 [==============================] - 7s 17ms/step - loss: 0.3829 - binary_accuracy: 0.8287 - f1_score: 0.7898
Epoch 3/10
424/424 [==============================] - 7s 16ms/step - loss: 0.3571 - binary_accuracy: 0.8423 - f1_score: 0.8076
Epoch 4/10
424/424 [==============================] - 8s 18ms/step - loss: 0.3340 - binary_accuracy: 0.8537 - f1_score: 0.8240
Epoch 5/10
424/424 [==============================] - 7s 16ms/step - loss: 0.3131 - binary_accuracy: 0.8645 - f1_score: 0.8358
Epoch 6/10
424/424 [==============================] - 7s 16ms/step - loss: 0.2964 - binary_accuracy: 0.8702 - f1_score: 0.8467
Epoch 7/10
424/424 [==============================] - 7s 15ms/step - loss: 0.2825 - binary_accuracy: 0.8782 - f1_score: 0.8542
Epoch 8/10
424/424 [==============================] - 7s 16ms/step - loss: 0.2693 - binary_accuracy: 0.8868 - f1_score: 0

## Evaluation on the test set

In [12]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

if NUM_CLASSES > 2:
    test_dset = test_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
test_ds =  test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [13]:
model.evaluate(test_ds)

142/142 [==============================] - 3s 16ms/step - loss: 0.3195 - binary_accuracy: 0.8646 - f1_score: 0.8443


[0.31945326924324036, 0.8646225333213806, 0.8443149924278259]